In [1]:
import sys
sys.path.append("./src")

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from prosit.simulator import SimulatorParameters, SimulatorEngine

import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
import pandas as pd

In [4]:
# import event log and Petri net model

log = xes_importer.apply("data/logs/purchasing.xes")
net, im, fm = pm4py.read_pnml("data/models/purchasing.pnml")

parsing log, completed traces :: 100%|██████████| 608/608 [00:00<00:00, 1547.38it/s]


In [5]:
# function for creating a recommendation DataFrame to use for simulation

def build_recommender_df(prev_log: pd.DataFrame, recommendations: dict) -> pd.DataFrame:

    prev_log["recommendation:act"] = None
    prev_log["recommendation:res"] = None 

    for case_id, recs in recommendations.items():
        act = recs.get("act", None)
        res = recs.get("res", None)
        last_index = prev_log[prev_log['case:concept:name'] == case_id].index[-1]
        prev_log.at[last_index, "recommendation:act"] = act
        prev_log.at[last_index, "recommendation:res"] = res
        
    return prev_log

In [6]:
# Create a random log with previous event (for test)

df_log = pm4py.convert_to_dataframe(log)
prev_log = df_log.iloc[:100]

In [7]:
prev_log

,start:timestamp,time:timestamp,concept:name,org:resource,org:role,case:concept:name
0,2011-01-01 00:00:00+00:00,2011-01-01 00:37:00+00:00,Create Purchase Requisition,Kim Passa,Requester,1
1,2011-01-01 05:37:00+00:00,2011-01-01 05:45:00+00:00,Create Request for Quotation Requester,Kim Passa,Requester,1
2,2011-01-01 06:41:00+00:00,2011-01-01 06:55:00+00:00,Analyze Request for Quotation,Karel de Groot,Purchasing Agent,1
3,2011-01-01 11:43:00+00:00,2011-01-01 12:09:00+00:00,Send Request for Quotation to Supplier,Karel de Groot,Purchasing Agent,1
4,2011-01-01 12:32:00+00:00,2011-01-01 16:03:00+00:00,Create Quotation comparison Map,Magdalena Predutta,Purchasing Agent,1
...,...,...,...,...,...,...
95,2011-01-01 11:20:00+00:00,2011-01-01 11:37:00+00:00,Create Purchase Requisition,Christian Francois,Requester,6
96,2011-01-01 17:32:00+00:00,2011-01-01 17:45:00+00:00,Create Request for Quotation Requester,Alberto Duport,Requester,6
97,2011-01-01 18:39:00+00:00,2011-01-01 18:55:00+00:00,Analyze Request for Quotation,Magdalena Predutta,Purchasing Agent,6
98,2011-01-01 19:47:00+00:00,2011-01-01 19:55:00+00:00,Amend Request for Quotation Requester,Penn Osterwalder,Requester Manager,6


In [8]:
# create a recommendation dictionary
# recommend activity "Amend Request for Quotation Requester Manager" and resource "Penn Osterwalder" for case "6"

recommendations = {
    "6": {
        "act": "Amend Request for Quotation Requester Manager", 
        "res": "Penn Osterwalder"
        }
    }

# build recommendation dataframe
log_rec = build_recommender_df(prev_log, recommendations)

In [9]:
# show recommendation DataFrame

log_rec

,start:timestamp,time:timestamp,concept:name,org:resource,org:role,case:concept:name,recommendation:act,recommendation:res
0,2011-01-01 00:00:00+00:00,2011-01-01 00:37:00+00:00,Create Purchase Requisition,Kim Passa,Requester,1,None,None
1,2011-01-01 05:37:00+00:00,2011-01-01 05:45:00+00:00,Create Request for Quotation Requester,Kim Passa,Requester,1,None,None
2,2011-01-01 06:41:00+00:00,2011-01-01 06:55:00+00:00,Analyze Request for Quotation,Karel de Groot,Purchasing Agent,1,None,None
3,2011-01-01 11:43:00+00:00,2011-01-01 12:09:00+00:00,Send Request for Quotation to Supplier,Karel de Groot,Purchasing Agent,1,None,None
4,2011-01-01 12:32:00+00:00,2011-01-01 16:03:00+00:00,Create Quotation comparison Map,Magdalena Predutta,Purchasing Agent,1,None,None
...,...,...,...,...,...,...,...,...
95,2011-01-01 11:20:00+00:00,2011-01-01 11:37:00+00:00,Create Purchase Requisition,Christian Francois,Requester,6,None,None
96,2011-01-01 17:32:00+00:00,2011-01-01 17:45:00+00:00,Create Request for Quotation Requester,Alberto Duport,Requester,6,None,None
97,2011-01-01 18:39:00+00:00,2011-01-01 18:55:00+00:00,Analyze Request for Quotation,Magdalena Predutta,Purchasing Agent,6,None,None
98,2011-01-01 19:47:00+00:00,2011-01-01 19:55:00+00:00,Amend Request for Quotation Requester,Penn Osterwalder,Requester Manager,6,None,None


In [10]:
# build simulator parameters and simulator engine

sim_params = SimulatorParameters(net, im, fm)
sim_params.discover_from_eventlog(log)

sim_engine = SimulatorEngine(sim_params)

Feature discovery...


100%|██████████| 608/608 [00:07<00:00, 80.16it/s] 


Transition probabilities discovery...


100%|██████████| 27/27 [00:00<00:00, 68.12it/s]


Resources discovery...
Calendars discovery...
Execution Time discovery...


100%|██████████| 24/24 [00:02<00:00,  9.05it/s]


Waiting Time discovery...


100%|██████████| 24/24 [00:02<00:00, 10.04it/s]


Arrival Time discovery...


In [11]:
# simulate event log with recommendations from previous log

sim_log = sim_engine.apply(prev_log=log_rec)

Simulating Cases: 100%|██████████| 3/3 [00:00<00:00, 118.17it/s]


In [12]:
# show complete event log: previous + simulated events

sim_log

,case:concept:name,concept:name,org:resource,start:timestamp,time:timestamp
0,1,Create Purchase Requisition,Kim Passa,2011-01-01 00:00:00+00:00,2011-01-01 00:37:00+00:00
1,2,Create Purchase Requisition,Immanuel Karagianni,2011-01-01 00:16:00+00:00,2011-01-01 00:29:00+00:00
2,3,Create Purchase Requisition,Kim Passa,2011-01-01 02:23:00+00:00,2011-01-01 03:03:00+00:00
3,1,Create Request for Quotation Requester,Kim Passa,2011-01-01 05:37:00+00:00,2011-01-01 05:45:00+00:00
4,1,Analyze Request for Quotation,Karel de Groot,2011-01-01 06:41:00+00:00,2011-01-01 06:55:00+00:00
...,...,...,...,...,...
149,6,Approve Purchase Order for payment,Magdalena Predutta,2011-02-01 19:52:00+00:00,2011-02-01 19:52:00+00:00
150,6,Send invoice,Carmen Finacse,2011-02-08 07:59:00+00:00,2011-02-08 07:59:00+00:00
151,6,Release Supplier's Invoice,Pedro Alvares,2011-02-08 22:46:00+00:00,2011-02-08 22:48:00+00:00
152,6,Authorize Supplier's Invoice payment,Pedro Alvares,2011-02-08 22:58:00+00:00,2011-02-08 22:58:00+00:00


In [13]:
# show simulated events for case "6"

sim_log[sim_log['case:concept:name'] == "6"]

,case:concept:name,concept:name,org:resource,start:timestamp,time:timestamp
11,6,Create Purchase Requisition,Christian Francois,2011-01-01 11:20:00+00:00,2011-01-01 11:37:00+00:00
18,6,Create Request for Quotation Requester,Alberto Duport,2011-01-01 17:32:00+00:00,2011-01-01 17:45:00+00:00
19,6,Analyze Request for Quotation,Magdalena Predutta,2011-01-01 18:39:00+00:00,2011-01-01 18:55:00+00:00
22,6,Amend Request for Quotation Requester,Penn Osterwalder,2011-01-01 19:47:00+00:00,2011-01-01 19:55:00+00:00
24,6,Analyze Request for Quotation,Francois de Perrier,2011-01-01 20:55:00+00:00,2011-01-01 21:28:00+00:00
41,6,Amend Request for Quotation Requester Manager,Penn Osterwalder,2011-01-02 03:32:00+00:00,2011-01-02 03:45:00+00:00
125,6,Analyze Request for Quotation,Francois de Perrier,2011-01-12 03:50:00+00:00,2011-01-12 04:17:00+00:00
132,6,Send Request for Quotation to Supplier,Magdalena Predutta,2011-01-17 10:03:00+00:00,2011-01-17 10:19:00+00:00
133,6,Create Quotation comparison Map,Francois de Perrier,2011-01-19 02:33:00+00:00,2011-01-19 05:46:00+00:00
134,6,Analyze Quotation comparison Map,Immanuel Karagianni,2011-01-20 01:23:00+00:00,2011-01-20 01:52:00+00:00
